### Assessing Convergence using Raftery & Lewis and Gweke Diagnostics 

This is an R script to assess convergence of sampling chains using the two diagnostics. 

In [ ]:
# Require packages 
library(coda)
library(graphics)

In [103]:
# Loading functions 

ACHROPTGPplots <- function(algoname,modelname,allALGO,ind){
    namePDF = paste("Results",algoname,"_",modelname,"_AutocorrTrace",sep="")
    pdf(namePDF)
    T = c("T100","T1000","T10000","T100000")
    counter = 1 
    for (algo in allALGO){   #list(OPTGP_A,OPTGP_B,OPTGP_C),OPTGP_D
        print(T[counter])
        x=algo[,ind] #"V549"
        MCMC = as.mcmc(x)
        acf(x,main=T[counter])
        traceplot(MCMC)
        print("Raftery & Lewis")
        rl = raftery.diag(MCMC, q=0.025, r=0.005, s=0.95, converge.eps=0.001)
        print(rl$resmatrix)
        print("Gweke")
        gw = geweke.diag(MCMC, frac1=0.1, frac2=0.5)
        print(gw$z)
        counter = counter + 1 
    }
    dev.off()
    print("Plot Saved!")
}

ALLcalcs <- function(allALGOA,allALGOB,allALGOC){
    T = c("T100","T1000","T10000","T100000")
    counter = 1 
    for (allALGO in list(allALGOA,allALGOB,allALGOC)){
        lenrlN_fails = c()
        lenrlI_fails = c()
        lengw_fails = c()
        for(algo in allALGO){ #list(OPTGP_1A,OPTGP_2A,OPTGP_3A)
            rlN_fail = c()
            rlI_fail = c()
            gw_fail = c()
            for (i in names(algo)){
                #print(i)
                x=algo[,i]
                MCMC = as.mcmc(x)
                if (length(unique(MCMC))>1){
                    rl = raftery.diag(MCMC, q=0.025, r=0.005, s=0.95, converge.eps=0.001)
                    rlN_fail = c(rlN_fail,rl$resmatrix[2]) # not enough samples 
                    if (rl$resmatrix[4]>5){rlI_fail = c(rlI_fail,rl$resmatrix[4])} # too sticky 
                    gw = geweke.diag(MCMC, frac1=0.1, frac2=0.5)
                    #print(abs(gw$z))
                    if((abs(gw$z) == Inf) | (is.nan(gw$z) == TRUE)){} else{
                        if (abs(gw$z) > 1.28){gw_fail = c(gw_fail,abs(gw$z))}
                    }
                }
            }
            lenrlN_fails = c(lenrlN_fails,max(rlN_fail))
            lenrlI_fails = c(lenrlI_fails,length(rlI_fail))
            lengw_fails = c(lengw_fails,length(gw_fail))
        }
        print(T[counter])
        print(paste("R&L N Max:",max(lenrlN_fails),"SD",sd(lenrlN_fails)))
        print(paste("R&L I Fail:",mean(lenrlI_fails),"SD",sd(lenrlI_fails)))
        print(paste("R&L I Fail:",mean(lengw_fails),"SD",sd(lengw_fails)))
        print(lenrlN_fails)
        print(lenrlI_fails)
        print(lengw_fails)
    }
    counter = counter + 1 
}

In [104]:
# Loading all of the Sampling Chains 
ALGO_1A = read.csv("./ACHR/ResultsACHR_Poolman2009_A_N5000_T100.csv",header=FALSE)
ALGO_1B = read.csv("./ACHR/ResultsACHR_Poolman2009_A_N5000_T1000.csv",header=FALSE)
ALGO_1C = read.csv("./ACHR/ResultsACHR_Poolman2009_A_N5000_T10000.csv",header=FALSE)
ALGO_2A = read.csv("./ACHR/ResultsACHR_Poolman2009_B_N5000_T100.csv",header=FALSE)
ALGO_2B = read.csv("./ACHR/ResultsACHR_Poolman2009_B_N5000_T1000.csv",header=FALSE)
ALGO_2C = read.csv("./ACHR/ResultsACHR_Poolman2009_B_N5000_T10000.csv",header=FALSE)
ALGO_3A = read.csv("./ACHR/ResultsACHR_Poolman2009_C_N5000_T100.csv",header=FALSE)
ALGO_3B = read.csv("./ACHR/ResultsACHR_Poolman2009_C_N5000_T1000.csv",header=FALSE)
ALGO_3C = read.csv("./ACHR/ResultsACHR_Poolman2009_C_N5000_T10000.csv",header=FALSE)

In [105]:
# Combining the sampling chains for analysis 
ALGOA = list(ALGO_1A,ALGO_2A,ALGO_3A)
ALGOB = list(ALGO_1B,ALGO_2B,ALGO_3B)
ALGOC = list(ALGO_1C,ALGO_2C,ALGO_3C)

ACHROPTGPplots("ACHR","Poolman2009",list(ALGO_1A,ALGO_1B,ALGO_1C),"V205")
ALLcalcs(ALGOA,ALGOB,ALGOC)

[1] "T100"
[1] "Raftery & Lewis"
      M     N Nmin    I
[1,] 10 11318 3746 3.02
[1] "Gweke"
     var1 
-2.719012 
[1] "T1000"
[1] "Raftery & Lewis"
     M    N Nmin    I
[1,] 3 4267 3746 1.14
[1] "Gweke"
     var1 
-1.072774 
[1] "T10000"
[1] "Raftery & Lewis"
     M    N Nmin   I
[1,] 3 4484 3746 1.2
[1] "Gweke"
      var1 
-0.8828681 
[1] "Plot Saved!"
[1] "T100"
[1] "R&L N Max: 2862482 SD 950904.576823563"
[1] "R&L I Fail: 744.333333333333 SD 9.29157324317757"
[1] "R&L I Fail: 394 SD 25.5342906696074"
[1] 2862482 1066918 2507490
[1] 752 747 734
[1] 416 400 366
[1] "T100"
[1] "R&L N Max: 1557990 SD 357950.785051521"
[1] "R&L I Fail: 647.666666666667 SD 5.13160143944688"
[1] "R&L I Fail: 322.333333333333 SD 7.09459888459759"
[1]  992682  894933 1557990
[1] 652 642 649
[1] 321 330 316
[1] "T100"
[1] "R&L N Max: 1083360 SD 235301.482293107"
[1] "R&L I Fail: 573 SD 7.93725393319377"
[1] "R&L I Fail: 283 SD 26.0576284415908"
[1]  619626 1083360  920872
[1] 579 576 564
[1] 281 310 258


In [47]:
# Generating Trace and ACF plots for one model and different thinning options 
pdf("ResultsCHRR_Poolman2009_C_N5000_AutocorrTrace.pdf")
print("Poolman2009 - CHRR")
T = c("T100","T1000","T10000","T100000")
counter = 1 
for (algo in list(ALGO_A,ALGO_B,ALGO_C)){ 
    print(T[counter])
    rlN_fail = c()
    rlI_fail = c()
    gw_fail = c()
    print(dim(algo))
    x=algo[205,]
    MCMC = as.mcmc(x)
    acf(x,main=T[counter])
    traceplot(MCMC)
    print("Raftery & Lewis")
    rl = raftery.diag(MCMC, q=0.025, r=0.005, s=0.95, converge.eps=0.001)
    print(rl$resmatrix)
    print("Gweke")
    gw = geweke.diag(MCMC, frac1=0.1, frac2=0.5)
    print(gw$z)
    counter = counter + 1
    for (i in 1:dim(OPTGP_A)[1]){
        x=algo[i,]
        MCMC = as.mcmc(x)
        if (length(unique(MCMC))>1){
            rl = raftery.diag(MCMC, q=0.025, r=0.005, s=0.95, converge.eps=0.001)
            if (rl$resmatrix[2]>5000){rlN_fail = c(rlN_fail,i)} # not enough samples 
            if (rl$resmatrix[4]>5){rlI_fail = c(rlI_fail,i)} # too sticky 
            gw = geweke.diag(MCMC, frac1=0.1, frac2=0.5)
            if (abs(gw$z) > 1.28){gw_fail = c(gw_fail,i)}
        }
    }
    print(length(rlN_fail))
    print(length(rlI_fail))
    print(length(gw_fail))
    print(" ")
}
dev.off()

[1] "Arnold - CHRR"
[1] "T100"
[1]  549 5000
[1] "Raftery & Lewis"
     M    N Nmin    I
[1,] 4 4792 3746 1.28
[1] "Gweke"
      var1 
-0.5677972 
[1] 429
[1] 117
[1] 154
[1] " "
[1] "T1000"
[1]  549 5000
[1] "Raftery & Lewis"
     M    N Nmin    I
[1,] 2 3930 3746 1.05
[1] "Gweke"
     var1 
0.8895393 
[1] 43
[1] 0
[1] 122
[1] " "
[1] "T10000"
[1]  549 5000
[1] "Raftery & Lewis"
     M    N Nmin     I
[1,] 2 3741 3746 0.999
[1] "Gweke"
    var1 
-1.44668 
[1] 0
[1] 0
[1] 146
[1] " "
[1] "T100000"
[1]  549 5000
[1] "Raftery & Lewis"
     M    N Nmin     I
[1,] 2 3620 3746 0.966
[1] "Gweke"
     var1 
-1.773557 
[1] 0
[1] 0
[1] 117
[1] " "


png 
  2

In [66]:
# Generting Trace and ACF plots for the target reactions of each of the models using sampling chains 
print("Raftery Lewis and Gweke for CHRR")
pdf("ResultsCHRR_A_N5000_AutocorrTrace.pdf")
modelnames = c("Arnold", "Poolman", "Gomes")
targetreacs = c(549,205,48)
counter = 1
CHRR_A = t(read.csv(paste('./CHRR/ResultsCHRR_Arnold2014_A_N5000.csv',sep = ""),header=FALSE))
CHRR_B = t(read.csv(paste('./CHRR/ResultsCHRR_Poolman2009_A_N5000.csv',sep = ""),header=FALSE))
CHRR_C = read.csv(paste('./CHRR/ResultsCHRR_Gomes2010_A_N5000.csv',sep = ""),header=FALSE)
for (CHRR in list(CHRR_Arnold, CHRR_Poolman, CHRR_Gomes)){
    x = CHRR[,targetreacs[counter]]
    MCMC = as.mcmc(x)
    acf(x,main=modelnames[counter])
    traceplot(MCMC)
    print(modelnames[counter])
    print(raftery.diag(MCMC, q=0.025, r=0.005, s=0.95, converge.eps=0.001))
    print(geweke.diag(MCMC, frac1=0.1, frac2=0.5))
    rlN_fail = c()
    rlI_fail = c()
    gw_fail = c()
    for (i in 1:dim(CHRR)[2]){
        x=CHRR[,i]
        MCMC = as.mcmc(x)
        if (length(unique(MCMC))>1){
            rl = raftery.diag(MCMC, q=0.025, r=0.005, s=0.95, converge.eps=0.001)
            if (rl$resmatrix[2]>5000){rlN_fail = c(rlN_fail,i)} # not enough samples 
            if (rl$resmatrix[4]>5){rlI_fail = c(rlI_fail,i)} # too sticky 
            gw = geweke.diag(MCMC, frac1=0.1, frac2=0.5)
            if (abs(gw$z) > 1.28){gw_fail = c(gw_fail,i)}
        }
    }
    counter = counter +1
    print(length(rlN_fail))
    print(length(rlI_fail))
    print(length(gw_fail))
}
dev.off()

[1] "Raftery Lewis and Gweke for CHRR"
[1] "Arnold"

Quantile (q) = 0.025
Accuracy (r) = +/- 0.005
Probability (s) = 0.95 
                                       
 Burn-in  Total Lower bound  Dependence
 (M)      (N)   (Nmin)       factor (I)
 2        3680  3746         0.982     


Fraction in 1st window = 0.1
Fraction in 2nd window = 0.5 

  var1 
-1.266 

[1] 0
[1] 0
[1] 87
[1] "Poolman"

Quantile (q) = 0.025
Accuracy (r) = +/- 0.005
Probability (s) = 0.95 
                                       
 Burn-in  Total Lower bound  Dependence
 (M)      (N)   (Nmin)       factor (I)
 2        3741  3746         0.999     


Fraction in 1st window = 0.1
Fraction in 2nd window = 0.5 

 var1 
1.005 

[1] 0
[1] 0
[1] 139
[1] "Gomes"

Quantile (q) = 0.025
Accuracy (r) = +/- 0.005
Probability (s) = 0.95 
                                       
 Burn-in  Total Lower bound  Dependence
 (M)      (N)   (Nmin)       factor (I)
 2        3680  3746         0.982     


Fraction in 1st window = 0.1
Fra

png 
  2

In [17]:
# all_modls = c('Arnold2014','Poolman2009','Gomes2010')
# all_ind = c('V549','V205','V47')
# for (i in 1:length(all_modls)){
#     modls = all_modls[i]
#     print(modls)
#     ind = all_ind[i]
#     #ACHR = read.csv(paste('ResultsACHR_',modls,'_B_N100000_T1000.csv',sep = ""),header=FALSE)
#     CHRR = read.csv(paste('./CHRR/ResultsCHRR_',modls,'_A_N5000.csv',sep = ""),header=FALSE)
#     #OPTGP = read.csv(paste('ResultsOPTGP_',modls,'_B_N100000_T1000.csv',sep = ""),header=FALSE)
#     # Raftery & Lewis Convergence Diagnostic
#     print("Testing Convergence")
#     for (alg in list(ACHR,CHRR,OPTGP)){
#         x=alg[0:5000,ind]
#         MCMC = as.mcmc(x)
#         print(raftery.diag(MCMC, q=0.025, r=0.005, s=0.95, converge.eps=0.001))
#     }
#     # Kruskal-Wallis Test 
#     print("ANOVA for Distribution Differences (Kruskal-Wallis)")
#     all_distr = c(ACHR[0:10000,ind],CHRR[0:10000,ind],OPTGP[0:10000,ind])
#     all_groups = as.factor(c(rep('ACHR',10000),rep('CHRR',10000),rep('OPTGP',10000)))
#     print(kruskal.test(all_distr,all_groups))
# }

[1] "Arnold2014"
[1] "Testing Convergence"

Quantile (q) = 0.025
Accuracy (r) = +/- 0.005
Probability (s) = 0.95 

You need a sample size of at least 3746 with these values of q, r and s
[1] "Poolman2009"
[1] "Testing Convergence"

Quantile (q) = 0.025
Accuracy (r) = +/- 0.005
Probability (s) = 0.95 

You need a sample size of at least 3746 with these values of q, r and s
[1] "Gomes2010"
[1] "Testing Convergence"

Quantile (q) = 0.025
Accuracy (r) = +/- 0.005
Probability (s) = 0.95 


ERROR: Error in if (x$resmatrix[1] == "Error") cat("\nYou need a sample size of at least", : missing value where TRUE/FALSE needed
